In [1]:
import os, shutil, random
from pathlib import Path

# 원본 데이터 경로들
SRC_DIRS = [
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725_fix",
    "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250721_fix",
]

# 결과 폴더
DEST_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/merge_1")

# 기존 결과 폴더 있으면 삭제
if DEST_DIR.exists():
    shutil.rmtree(DEST_DIR)
DEST_DIR.mkdir(parents=True)

# train/val/test 비율
RATIO = (0.8, 0.1, 0.1)
SPLITS = ["train", "val", "test"]

# split 디렉토리 생성
for split in SPLITS:
    (DEST_DIR / split / "images").mkdir(parents=True, exist_ok=True)
    (DEST_DIR / split / "labels").mkdir(parents=True, exist_ok=True)

# jpg/txt 수집
all_images = []
for src in SRC_DIRS:
    for img in Path(src).glob("*.jpg"):
        txt = img.with_suffix(".txt")
        if txt.exists():
            all_images.append((img, txt))

# 무작위 셔플
random.shuffle(all_images)

# 데이터 나누기
n = len(all_images)
train_end = int(n * RATIO[0])
val_end = train_end + int(n * RATIO[1])

splits_data = {
    "train": all_images[:train_end],
    "val": all_images[train_end:val_end],
    "test": all_images[val_end:],
}

# 복사
for split, pairs in splits_data.items():
    for img, txt in pairs:
        shutil.copy(img, DEST_DIR / split / "images" / img.name)
        shutil.copy(txt, DEST_DIR / split / "labels" / txt.name)

print("총 이미지 수:", n)
for s, p in splits_data.items():
    print(f"{s}: {len(p)}개")


총 이미지 수: 4142
train: 3313개
val: 414개
test: 415개


In [2]:
from pathlib import Path
from collections import Counter

BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/merge_1")
SPLITS = ["train", "val", "test"]

def count_split(split_dir: Path) -> Counter:
    c = Counter()
    for lbl in (split_dir / "labels").rglob("*.txt"):
        with open(lbl, "r", encoding="utf-8", errors="ignore") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5 and parts[0].isdigit():
                    c[int(parts[0])] += 1
    return c

# 전체 카운트
total_counter = Counter()
for split in SPLITS:
    c = count_split(BASE / split)
    total_counter.update(c)
    print(f"{split} → {dict(c)} (총 {sum(c.values())} 개)")

print("\n=== 전체 합계 ===")
for cls_id, cnt in sorted(total_counter.items()):
    print(f"class {cls_id}: {cnt}")
print("총 박스:", sum(total_counter.values()))


train → {1: 3927, 0: 2281, 2: 254} (총 6462 개)
val → {1: 448, 0: 319, 2: 33} (총 800 개)
test → {0: 290, 1: 491, 2: 23} (총 804 개)

=== 전체 합계 ===
class 0: 2890
class 1: 4866
class 2: 310
총 박스: 8066


In [6]:
import os, math, random
from pathlib import Path
from collections import defaultdict, Counter
import cv2
import albumentations as A

# =========================
# 설정
# =========================
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/merge_1/train")
IMG_DIR = BASE / "images"
LBL_DIR = BASE / "labels"

OUT_IMG = BASE / "images_aug"
OUT_LBL = BASE / "labels_aug"
OUT_IMG.mkdir(parents=True, exist_ok=True)
OUT_LBL.mkdir(parents=True, exist_ok=True)

TARGET_N = 1500         # 만들 증강 샘플 수
RANDOM_SEED = 42        # 재현성
MIN_WH_PRE = 1e-6       # 사전 클리핑 시 최소 폭/높이
MIN_WH_POST = 0.005     # 증강 후 너무 작은 박스 제거 기준

# =========================
# 유틸
# =========================
def read_yolo_label(p: Path):
    """YOLO 라벨 읽기 -> [(cls, x, y, w, h), ...]"""
    rows = []
    if not p.exists():
        return rows
    with open(p, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5 and parts[0].isdigit():
                cls = int(parts[0])
                x, y, w, h = map(float, parts[1:5])
                rows.append([cls, x, y, w, h])
    return rows

def write_yolo_label(p: Path, rows):
    with open(p, "w", encoding="utf-8") as f:
        for cls, x, y, w, h in rows:
            f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

def only_class0(rows):
    """해당 이미지 라벨이 모두 class 0이고, 최소 1개 박스"""
    return len(rows) > 0 and all(r[0] == 0 for r in rows)

# 변환 전/후 박스 경계 안정화
EPS = 1e-9
def yolo_clip_and_filter(boxes, min_wh=1e-6):
    """
    boxes: [[xc,yc,w,h], ...] in [0,1] approx
    1) corners로 변환→클리핑→다시 yolo로
    2) 너무 작은 박스(폭/높이 < min_wh) 제거
    """
    kept = []
    for (xc, yc, w, h) in boxes:
        x1 = xc - w/2
        y1 = yc - h/2
        x2 = xc + w/2
        y2 = yc + h/2

        x1 = max(0.0, min(1.0, x1))
        y1 = max(0.0, min(1.0, y1))
        x2 = max(0.0, min(1.0, x2))
        y2 = max(0.0, min(1.0, y2))

        w2 = x2 - x1
        h2 = y2 - y1
        if w2 < min_wh or h2 < min_wh:
            continue

        xc2 = (x1 + x2) / 2.0
        yc2 = (y1 + y2) / 2.0

        xc2 = max(0.0, min(1.0, xc2))
        yc2 = max(0.0, min(1.0, yc2))
        w2 = max(EPS, min(1.0, w2))
        h2 = max(EPS, min(1.0, h2))
        kept.append([xc2, yc2, w2, h2])
    return kept

# =========================
# 증강 파이프라인 (bbox 안전)
# =========================
bbox_params = A.BboxParams(
    format="yolo",
    label_fields=["class_labels"],
    min_visibility=0.0,   # 내부 드롭 최소화 → 후처리에서 직접 필터
)

pipelines = [
    A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.3),
        A.Affine(scale=(0.9,1.1), translate_percent=(-0.05,0.05),
                 rotate=(-5,5), shear=(-3,3),
                 fit_output=False, p=0.8,
                 mode=cv2.BORDER_REFLECT_101),
        A.MotionBlur(blur_limit=5, p=0.2),
        A.RandomGamma(p=0.2),
    ], bbox_params=bbox_params),

    A.Compose([
        A.CLAHE(p=0.4),
        A.Affine(scale=(0.95,1.05), translate_percent=(-0.08,0.08),
                 rotate=(-7,7), shear=(-4,4),
                 fit_output=False, p=0.9,
                 mode=cv2.BORDER_REFLECT_101),
        A.GaussNoise(var_limit=(5.0, 25.0), p=0.3),
        A.RandomBrightnessContrast(p=0.4),
    ], bbox_params=bbox_params),

    A.Compose([
        A.Affine(scale=(0.9,1.1), translate_percent=(-0.1,0.1),
                 rotate=(-8,8), shear=(-5,5),
                 fit_output=False, p=1.0,
                 mode=cv2.BORDER_REFLECT_101),
        A.RandomFog(fog_coef_lower=0.02, fog_coef_upper=0.08, p=0.2),
        A.RandomSunFlare(src_radius=80, num_flare_circles_lower=3, p=0.15),
    ], bbox_params=bbox_params),
]

# =========================
# 클래스 0-only 후보 수집
# =========================
candidates = []
for img_p in IMG_DIR.glob("*.jpg"):
    lbl_p = LBL_DIR / (img_p.stem + ".txt")
    rows = read_yolo_label(lbl_p)
    if only_class0(rows):
        candidates.append((img_p, lbl_p))

if not candidates:
    raise SystemExit("클래스 0만 포함된 train 데이터가 없습니다.")

print(f"[INFO] 클래스0-only 원본 이미지 수: {len(candidates)}")

# =========================
# 증강 루프
# =========================
random.seed(RANDOM_SEED)
rng = random.Random(RANDOM_SEED)
rng.shuffle(candidates)

per_src_cap = max(2, math.ceil(TARGET_N / len(candidates)))  # 원본별 상한
per_src_done = defaultdict(int)
made = 0
skipped_empty = 0
attempts = 0
max_attempts = TARGET_N * 8  # 안전장치

while made < TARGET_N and attempts < max_attempts:
    attempts += 1
    img_p, lbl_p = candidates[attempts % len(candidates)]
    if per_src_done[img_p] >= per_src_cap:
        continue

    img = cv2.imread(str(img_p))
    if img is None:
        continue

    rows = read_yolo_label(lbl_p)
    if not rows:
        continue

    # --- 1) 증강 전에 사전 클리핑/정제 (미세 음수/초과 제거) ---
    boxes_yolo = [[r[1], r[2], r[3], r[4]] for r in rows]
    boxes_yolo = yolo_clip_and_filter(boxes_yolo, min_wh=MIN_WH_PRE)
    if len(boxes_yolo) == 0:
        skipped_empty += 1
        continue
    classes = [0] * len(boxes_yolo)  # 전부 class 0

    # --- 2) 증강 ---
    aug = rng.choice(pipelines)
    out = aug(image=img, bboxes=boxes_yolo, class_labels=classes)
    aug_img = out["image"]
    aug_boxes = out["bboxes"]          # [[xc,yc,w,h], ...]
    aug_cls = out["class_labels"]

    # --- 3) 증강 후 2차 클리핑/필터 ---
    aug_boxes = yolo_clip_and_filter(aug_boxes, min_wh=MIN_WH_POST)
    if len(aug_boxes) == 0:
        skipped_empty += 1
        continue

    # 저장
    aug_name = f"{img_p.stem}_c0aug{per_src_done[img_p]:04d}"
    out_img_p = OUT_IMG / f"{aug_name}.jpg"
    out_lbl_p = OUT_LBL / f"{aug_name}.txt"

    cv2.imwrite(str(out_img_p), aug_img)
    merged = [[int(aug_cls[i]), *aug_boxes[i]] for i in range(len(aug_boxes))]
    write_yolo_label(out_lbl_p, merged)

    per_src_done[img_p] += 1
    made += 1

# =========================
# 결과 리포트
# =========================
print("\n=== 증강 완료 ===")
print(f"생성된 증강 이미지: {made}")
print(f"스킵(박스 소실/클립 후 0 등): {skipped_empty}")
print("소스별 생성 개수 상위 10:")
for k, v in sorted(per_src_done.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"- {k.name}: {v}")

# 간단한 라벨 유효성 점검
def check_counts(dir_lbl: Path):
    cnt = Counter()
    files = 0
    for lbl in dir_lbl.glob("*.txt"):
        rows = read_yolo_label(lbl)
        if rows:
            files += 1
            for r in rows:
                cnt[r[0]] += 1
    return files, cnt

files_aug, cnt_aug = check_counts(OUT_LBL)
print(f"\n증강 라벨 파일 수: {files_aug}")
print("증강 박스 클래스 분포:", dict(sorted(cnt_aug.items())))



/tmp/ipykernel_8865/4063804145.py:99: UserWarning: Argument(s) 'mode' are not valid for transform Affine
  A.Affine(scale=(0.9,1.1), translate_percent=(-0.05,0.05),
/tmp/ipykernel_8865/4063804145.py:109: UserWarning: Argument(s) 'mode' are not valid for transform Affine
  A.Affine(scale=(0.95,1.05), translate_percent=(-0.08,0.08),
/tmp/ipykernel_8865/4063804145.py:113: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 25.0), p=0.3),
/tmp/ipykernel_8865/4063804145.py:118: UserWarning: Argument(s) 'mode' are not valid for transform Affine
  A.Affine(scale=(0.9,1.1), translate_percent=(-0.1,0.1),
/tmp/ipykernel_8865/4063804145.py:122: UserWarning: Argument(s) 'fog_coef_lower, fog_coef_upper' are not valid for transform RandomFog
  A.RandomFog(fog_coef_lower=0.02, fog_coef_upper=0.08, p=0.2),
/tmp/ipykernel_8865/4063804145.py:123: UserWarning: Argument(s) 'num_flare_circles_lower' are not valid for transform RandomSunFlare
  A.Random

[INFO] 클래스0-only 원본 이미지 수: 861

=== 증강 완료 ===
생성된 증강 이미지: 1500
스킵(박스 소실/클립 후 0 등): 28
소스별 생성 개수 상위 10:
- 20250721_053640_0021_000107_f001060_1920x1536_1219.jpg: 2
- 20250721_053640_0021_000008_f000070_1920x1536_967.jpg: 2
- 20250721_053640_0029_000029_f000280_1920x1536_2351.jpg: 2
- 20250721_053640_0021_000025_f000240_1920x1536_1017.jpg: 2
- 20250721_053640_0030_000174_f001730_1920x1536_3220.jpg: 2
- 20250725_171153_0000_000024_f000690_1920x1536.jpg: 2
- 20250725_172530_0008_000039_f001140_1920x1536.jpg: 2
- 20250725_172530_0012_000022_f000630_1920x1536.jpg: 2
- 20250725_172530_0007_000027_f000780_1920x1536.jpg: 2
- 20250721_053640_0007_000053_f000520_1920x1536_598.jpg: 2

증강 라벨 파일 수: 1500
증강 박스 클래스 분포: {0: 2179}


In [7]:
from pathlib import Path
from collections import Counter, defaultdict
import json

BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/merge_1")
SPLITS = ["train", "val", "test"]
ALLOWED = {0, 1, 2}          # 사용 중인 클래스 ID 집합
SAVE_LISTS = True            # 버킷별 파일목록 .txt 저장할지

def parse_label_file(p: Path):
    rows, invalid = [], 0
    try:
        with open(p, "r", encoding="utf-8", errors="ignore") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5 and parts[0].lstrip("-").isdigit():
                    try:
                        cls = int(parts[0]); x,y,w,h = map(float, parts[1:5])
                        rows.append((cls, x, y, w, h))
                    except Exception:
                        invalid += 1
                else:
                    if line.strip():
                        invalid += 1
    except FileNotFoundError:
        pass
    return rows, invalid

def analyze_group(img_dir: Path, lbl_dir: Path, tag: str):
    """한 그룹(예: train/orig, train/aug, val, test)을 분석"""
    class_counts = Counter()
    files_with_boxes = 0
    empty_label_files = 0
    invalid_lines = 0
    invalid_class_lines = 0
    invalid_class_ids = Counter()
    images_missing_label = 0
    labels_without_image = 0

    # per-image 버킷: class0-only / class1-only / class2-only / mixed / empty
    buckets = defaultdict(list)

    imgs = sorted(img_dir.glob("*.jpg"))
    lbls = sorted(lbl_dir.glob("*.txt"))

    image_basenames = {p.stem for p in imgs}
    label_basenames = {p.stem for p in lbls}

    images_missing_label = len(image_basenames - label_basenames)
    labels_without_image = len(label_basenames - image_basenames)

    for lbl in lbls:
        rows, inv = parse_label_file(lbl)
        invalid_lines += inv
        if rows:
            files_with_boxes += 1
            cls_set = set()
            for (cls, x, y, w, h) in rows:
                class_counts[cls] += 1
                cls_set.add(cls)
                if ALLOWED and cls not in ALLOWED:
                    invalid_class_lines += 1
                    invalid_class_ids[cls] += 1

            # 버킷 분류
            if len(cls_set) == 1:
                only = list(cls_set)[0]
                buckets[f"class{only}-only"].append(lbl.stem)
            else:
                buckets["mixed"].append(lbl.stem)
        else:
            empty_label_files += 1
            buckets["empty"].append(lbl.stem)

    summary = {
        "tag": tag,
        "images(.jpg)": len(imgs),
        "labels(.txt)": len(lbls),
        "files_with_boxes": files_with_boxes,
        "empty_label_files": empty_label_files,
        "images_missing_label": images_missing_label,
        "labels_without_image": labels_without_image,
        "invalid_lines": invalid_lines,
        "invalid_class_lines": invalid_class_lines,
        "invalid_class_ids": dict(invalid_class_ids),
        "class_counts": dict(sorted(class_counts.items())),
        "total_boxes": sum(class_counts.values()),
        "bucket_counts": {k: len(v) for k, v in buckets.items()},
    }

    # 선택적으로 리스트를 저장
    if SAVE_LISTS:
        out_dir = BASE / "_analysis_lists"
        out_dir.mkdir(parents=True, exist_ok=True)
        for k, names in buckets.items():
            out_file = out_dir / f"{tag}__{k}.txt"
            with open(out_file, "w", encoding="utf-8") as f:
                for n in sorted(names):
                    f.write(n + "\n")

    return summary

def print_summary(sumd):
    print(f"\n[{sumd['tag']}]")
    print(f"- images: {sumd['images(.jpg)']}  labels: {sumd['labels(.txt)']}")
    print(f"- files_with_boxes: {sumd['files_with_boxes']}  empty_label_files: {sumd['empty_label_files']}")
    print(f"- images_missing_label: {sumd['images_missing_label']}  labels_without_image: {sumd['labels_without_image']}")
    print(f"- invalid_lines: {sumd['invalid_lines']}  invalid_class_lines: {sumd['invalid_class_lines']}")
    if sumd["invalid_class_ids"]:
        print(f"- invalid_class_ids: {sumd['invalid_class_ids']}")
    print(f"- class_counts: {sumd['class_counts']}")
    print(f"- total_boxes: {sumd['total_boxes']}")
    print(f"- bucket_counts: {sumd['bucket_counts']}")

# ===== 실행: train은 orig/aug 분리, val/test는 orig만 =====
all_summaries = []

# train - original
train_orig = analyze_group(
    BASE / "train" / "images",
    BASE / "train" / "labels",
    tag="train_orig"
)
all_summaries.append(train_orig)

# train - augmented (존재하면)
train_aug = None
aug_img_dir = BASE / "train" / "images_aug"
aug_lbl_dir = BASE / "train" / "labels_aug"
if aug_img_dir.exists() and aug_lbl_dir.exists():
    train_aug = analyze_group(aug_img_dir, aug_lbl_dir, tag="train_aug")
    all_summaries.append(train_aug)

# val
val_sum = analyze_group(
    BASE / "val" / "images",
    BASE / "val" / "labels",
    tag="val"
)
all_summaries.append(val_sum)

# test
test_sum = analyze_group(
    BASE / "test" / "images",
    BASE / "test" / "labels",
    tag="test"
)
all_summaries.append(test_sum)

# 출력
for s in all_summaries:
    print_summary(s)

# 전체 합계(원본+증강 포함/미포함 둘 다 보고 싶을 수 있음)
from collections import Counter as C
tot_incl_aug, tot_excl_aug = C(), C()
for s in all_summaries:
    tot_incl_aug.update(s["class_counts"])
    if not s["tag"].endswith("_aug"):
        tot_excl_aug.update(s["class_counts"])

def show_total(title, c: C):
    print(f"\n=== {title} ===")
    d = dict(sorted(c.items()))
    print(d)
    total = sum(c.values())
    print("총 박스:", total)
    if total > 0:
        print("클래스 비율(%):", {k: f"{v/total*100:.2f}%" for k, v in d.items()})

show_total("전체 합계 (증강 포함)", tot_incl_aug)
show_total("전체 합계 (증강 제외: 원본만)", tot_excl_aug)



[train_orig]
- images: 3313  labels: 3313
- files_with_boxes: 3313  empty_label_files: 0
- images_missing_label: 0  labels_without_image: 0
- invalid_lines: 0  invalid_class_lines: 0
- class_counts: {0: 2281, 1: 3927, 2: 254}
- total_boxes: 6462
- bucket_counts: {'class1-only': 1643, 'mixed': 736, 'class0-only': 861, 'class2-only': 73}

[train_aug]
- images: 1500  labels: 1500
- files_with_boxes: 1500  empty_label_files: 0
- images_missing_label: 0  labels_without_image: 0
- invalid_lines: 0  invalid_class_lines: 0
- class_counts: {0: 2179}
- total_boxes: 2179
- bucket_counts: {'class0-only': 1500}

[val]
- images: 414  labels: 414
- files_with_boxes: 414  empty_label_files: 0
- images_missing_label: 0  labels_without_image: 0
- invalid_lines: 0  invalid_class_lines: 0
- class_counts: {0: 319, 1: 448, 2: 33}
- total_boxes: 800
- bucket_counts: {'class1-only': 191, 'mixed': 109, 'class0-only': 109, 'class2-only': 5}

[test]
- images: 415  labels: 415
- files_with_boxes: 415  empty_labe

In [8]:
# === run_me_once_merge_aug_into_train.py ===
from pathlib import Path
import shutil

BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/merge_1/train")
ORIG_IMG = BASE / "images"
ORIG_LBL = BASE / "labels"
AUG_IMG  = BASE / "images_aug"
AUG_LBL  = BASE / "labels_aug"

# 증강 폴더가 없으면 패스
if AUG_IMG.exists() and AUG_LBL.exists():
    cnt_i = cnt_l = 0
    for p in sorted(AUG_IMG.glob("*.jpg")):
        dst = ORIG_IMG / p.name
        if not dst.exists():  # 중복 방지
            shutil.copy(p, dst)
            cnt_i += 1
    for p in sorted(AUG_LBL.glob("*.txt")):
        dst = ORIG_LBL / p.name
        if not dst.exists():
            shutil.copy(p, dst)
            cnt_l += 1
    print(f"[OK] merged aug into train → +images {cnt_i}, +labels {cnt_l}")
else:
    print("[INFO] No aug folders found. Skipped.")


[OK] merged aug into train → +images 1500, +labels 1500


In [9]:
# === write_data_yaml.py ===
from pathlib import Path
import yaml

ROOT = "/home/dw/ws_job_msislab/Golf_Project/data/merge_1"

cfg = {
    "path": ROOT,   # Ultralytics가 내부 상대경로로 train/val/test 찾음
    "train": "train/images",
    "val":   "val/images",
    "test":  "test/images",
    "nc": 3,
    "names": ["Divot", "Fixed_Divot", "Diseased_Grass"],
}

out = Path(ROOT) / "data.yaml"
with open(out, "w", encoding="utf-8") as f:
    yaml.safe_dump(cfg, f, sort_keys=False, allow_unicode=True)
print(f"[OK] wrote {out}")

[OK] wrote /home/dw/ws_job_msislab/Golf_Project/data/merge_1/data.yaml


In [1]:
from ultralytics import YOLO

# 데이터셋 yaml 경로
DATA = "/home/dw/ws_job_msislab/Golf_Project/data/merge_1/data.yaml"

# 모델 불러오기 (pretrained yolov8s)
model = YOLO("yolov8s.pt")

# 학습 실행
model.train(
    data=DATA,
    epochs=300, patience=50,        # 총 200 epoch, 성능 정체시 50 epoch 조기 종료
    imgsz=896,                      # 입력 해상도
    batch=4,                        # 배치 크기 (GPU 메모리에 맞게 조절)
    device=0,                       # GPU:0 사용
    cache=True, workers=4,          # 캐싱 + dataloader 워커
    rect=True,                      # aspect-ratio 유지
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",  # 저장 상위 폴더
    name="merge1_exp1",             # 실험 이름 (폴더명)
    
    # ===== Data Augmentation =====
    mosaic=0.8, close_mosaic=10,    # Mosaic: 80% 확률, 마지막 10 epoch 비활성화
    mixup=0.1,                      # MixUp: 10% 확률
    erasing=0.25,                   # Cutout/Random Erasing 확률
    hsv_h=0.02, hsv_s=0.6, hsv_v=0.5,  # 색상/채도/밝기 변화
    label_smoothing=0.05,           # 레이블 스무딩 (5%)
    
    seed=2025                       # 재현성 고정
)


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/dw/ws_job_msislab/Golf_Project/data/merge_1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.25, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.6, hsv_v=0.5, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.8, multi_scale=False, name=merge1_exp1, nbs=64, nms=Fal

train: Scanning /home/dw/ws_job_msislab/Golf_Project/data/merge_1/train/labels.cache... 4813 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4813/4813 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (8.6GB RAM): 100%|██████████| 4813/4813 [00:11<00:00, 420.87it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 618.7±307.2 MB/s, size: 574.4 KB)


...

                   all        414        800      0.742      0.709      0.748      0.482



300 epochs completed in 6.480 hours.
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/merge1_exp1/weights/last.pt, 22.6MB
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/merge1_exp1/weights/best.pt, 22.6MB

Validating /home/dw/ws_job_msislab/Golf_Project/runs_yolo/merge1_exp1/weights/best.pt...
Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:02<00:00, 23.98it/s]


                   all        414        800      0.741      0.709      0.748      0.481
                 Divot        215        319      0.596      0.712      0.717      0.438
           Fixed_Divot        291        448      0.763      0.842      0.859      0.521
        Diseased_Grass         24         33      0.863      0.575       0.67      0.485
Speed: 0.3ms preprocess, 3.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/dw/ws_job_msislab/Golf_Project/runs_yolo/merge1_exp1


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa49c17f850>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [6]:
from ultralytics import YOLO
import numpy as np
import math

# === 경로와 모델 로드 ===
RUN_DIR = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/normal_train"
MODEL_PT = f"{RUN_DIR}/weights/best.pt"

model = YOLO(MODEL_PT)

# === 검증 실행 (data.yaml의 val split 사용) ===
metrics = model.val(data="/home/dw/ws_job_msislab/Golf_Project/data/merge_1/data.yaml")

box = metrics.box
names = model.names  # {class_id: class_name}

# --- 전체 지표 ---
print(f"[Overall] mAP50={box.map50:.4f}, mAP50-95={box.map:.4f}")

# --- 안전한 숫자 포매팅 헬퍼 ---
def fmt(x):
    try:
        if x is None or (isinstance(x, float) and (math.isnan(x) or math.isinf(x))):
            return "nan"
        return f"{x:.4f}"
    except Exception:
        return "nan"

# --- 클래스별 Precision/Recall ---
# p: precision per class, r: recall per class (길이 = nc)
precisions = getattr(box, "p", None)   # list[float] length=nc
recalls    = getattr(box, "r", None)   # list[float] length=nc
nc         = getattr(box, "nc", len(names))

# --- 클래스별 mAP50 계산 ---
# all_ap: shape (num_present_classes, 10)  IoU thresholds = 0.50..0.95 (10개)
# ap_class_index: 각 행이 어떤 class id인지
ap50_full = np.full(nc, np.nan, dtype=float)

if hasattr(box, "all_ap") and hasattr(box, "ap_class_index"):
    all_ap = np.array(box.all_ap)              # (k, 10)
    ap_cls = np.array(box.ap_class_index)      # (k,)
    if all_ap.ndim == 2 and all_ap.shape[1] >= 1:
        ap50 = all_ap[:, 0]                    # IoU=0.50 열
        ap50_full[ap_cls] = ap50

# --- 출력 ---
print("\n[Per-class metrics]")
print(f"{'class_id':>7s}  {'class_name':15s}  {'Precision':>10s}  {'Recall':>8s}  {'mAP50':>8s}")
for cid, cname in names.items():
    P = precisions[cid] if precisions is not None and cid < len(precisions) else float('nan')
    R = recalls[cid]    if recalls    is not None and cid < len(recalls)    else float('nan')
    AP50 = ap50_full[cid]
    print(f"{cid:7d}  {cname:15s}  {fmt(P):>10s}  {fmt(R):>8s}  {fmt(AP50):>8s}")



Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 8289.5±1251.9 MB/s, size: 1104.0 KB)


val: Scanning /home/dw/ws_job_msislab/Golf_Project/data/merge_1/val/labels.cache... 414 images, 0 backgrounds, 0 corrupt: 100%|██████████| 414/414 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:04<00:00,  5.64it/s]


                   all        414        800      0.922      0.649      0.724      0.648
                 DIVOT        215        319      0.878      0.555      0.611      0.536
           FIXED_DIVOT        291        448      0.895      0.846      0.927      0.851
                 GRASS         24         33      0.992      0.545      0.634      0.557
Speed: 0.6ms preprocess, 7.6ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val5
[Overall] mAP50=0.7242, mAP50-95=0.6478

[Per-class metrics]
class_id  class_name        Precision    Recall     mAP50
      0  DIVOT                0.8784    0.5549    0.6109
      1  FIXED_DIVOT          0.8954    0.8460    0.9273
      2  GRASS                0.9922    0.5455    0.6342
